***
# TEST
***

In [ ]:
import os
import json
import pandas as pd
import pvdeg
from pytest import approx
from pvdeg import TEST_DATA_DIR

#Load weather data
WEATHER = pd.read_csv(os.path.join(TEST_DATA_DIR, 'weather_day_pytest.csv'),
                         index_col= 0, parse_dates=True)
with open(os.path.join(TEST_DATA_DIR, 'meta.json'),'r') as file:
    META = json.load(file)

#Load expected results
rh_expected = pd.read_csv(os.path.join(TEST_DATA_DIR, 'input_day_pytest.csv'),
                          index_col=0, parse_dates=True)
rh_cols = [col for col in rh_expected.columns if 'RH' in col]
rh_expected = rh_expected[rh_cols]


def test_module():
    '''
    test pvdeg.humidity.calc_rel_humidity
    
    Requires:
    ---------
    weather dataframe and meta dictionary
    '''
    result = pvdeg.humidity.module(WEATHER, META)
    pd.testing.assert_frame_equal(result, 
                                  rh_expected, 
                                  check_dtype=False)

def test_psat():
    '''
    test pvdeg.humidity.psat
    
    Requires:
    ---------
    weahter dataframe and meta dictionary
    '''
    psat_avg = pvdeg.humidity.psat(temp=WEATHER['temp_air'])[1]
    assert psat_avg == approx(0.47607, abs=5e-5)



In [ ]:
test_psat()

In [1]:
import pvdeg
import pickle
import pandas as pd
import numpy as np
import xarray as xr
import os
from pvdeg import TEST_DATA_DIR

GEO_META = pd.read_csv(os.path.join(TEST_DATA_DIR, "summit-meta.csv"), index_col=0)
with open(os.path.join(TEST_DATA_DIR, "summit-weather.pkl"), 'rb') as f:
    GEO_WEATHER = pickle.load(f)

autotemplate

In [2]:
autotemplate_result = pvdeg.geospatial.auto_template(
    func=pvdeg.humidity.module, 
    ds_gids=GEO_WEATHER
).compute()

In [3]:
humidity_template = xr.open_dataset(os.path.join(TEST_DATA_DIR, 'humidity_template.nc')).compute()

In [ ]:
def compare_datasets(ds1: xr.Dataset, ds2: xr.Dataset, atol=1e-10) -> bool:
    """Compare loaded datasets with "empty-like" values"""

    if ds1.dims != ds2.dims:
        return False

    if set(ds1.coords.keys()) != set(ds2.coords.keys()):
        return False

    for coord in ds1.coords:
        if ds1.coords[coord].dtype.kind in {'i', 'f'}:
            # Use np.allclose for numeric coordinates
            if not np.allclose(ds1.coords[coord], ds2.coords[coord], atol=atol):
                return False
        elif ds1.coords[coord].dtype.kind == 'M':  # datetime64 type
            # Use array equality for datetime coordinates
            if not np.array_equal(ds1.coords[coord], ds2.coords[coord]):
                return False
        else:
            if not np.array_equal(ds1.coords[coord], ds2.coords[coord]):
                return False

    if set(ds1.data_vars.keys()) != set(ds2.data_vars.keys()):
        return False

    for var in ds1.data_vars:
        if not np.allclose(ds1[var], ds2[var], atol=atol):
            return False

    for dim in ds1.dims:
        if not ds1.indexes[dim].equals(ds2.indexes[dim]):
            return False

    return True

In [5]:
assert pvdeg.utilities.compare_datasets(autotemplate_result,humidity_template)

C:\Users\tford\dev\PVDegradationTools\pvdeg\utilities.py:479: FutureWarning: The return type of `Dataset.dims` will be changed to return a set of dimension names in future, in order to be more consistent with `DataArray.dims`. To access a mapping from dimension names to lengths, please use `Dataset.sizes`.
  if ds1.dims != ds2.dims:


output template

In [8]:
shapes = {
    "RH_surface_outside": ("gid", "time"),
    "RH_front_encap": ("gid", "time"),
    "RH_back_encap": ("gid", "time"),
    "RH_backsheet": ("gid", "time"),
}

manual_template = pvdeg.geospatial.output_template(
    shapes=shapes,
    ds_gids=GEO_WEATHER
).compute()

In [11]:
pvdeg.utilities.compare_datasets(manual_template, humidity_template)

C:\Users\tford\dev\PVDegradationTools\pvdeg\utilities.py:479: FutureWarning: The return type of `Dataset.dims` will be changed to return a set of dimension names in future, in order to be more consistent with `DataArray.dims`. To access a mapping from dimension names to lengths, please use `Dataset.sizes`.
  if ds1.dims != ds2.dims:


True

In [ ]:
# test template

shapes = {
    'testA' : ("gid", ),
    'testB' : ("gid", "time")
}

template = pvdeg.geospatial.output_template(
    shapes=shapes,

    ds_gids=GEO_WEATHER,
)


In [ ]:
template.to_netcdf(os.path.join(TEST_DATA_DIR, "mismatch-template.nc"))